In [44]:
from pynq import Overlay
import time
ol = Overlay('matrix.bit')

In [3]:
#ol?

In [45]:
dma = ol.axi_dma_0

In [46]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [47]:
import numpy as np
import random 

In [48]:
random.seed(2)
data_A = np.random.uniform(low=1, high=2, size=(8, 8)).astype(float)
data_B = np.random.uniform(low=1, high=2, size=(8, 8)).astype(float)

In [49]:
data_A, data_B

(array([[1.16763793, 1.76083613, 1.6171598 , 1.60581055, 1.97313797,
         1.49387104, 1.68515268, 1.11433389],
        [1.1055406 , 1.6582569 , 1.28133038, 1.52072141, 1.20111532,
         1.53132037, 1.73651009, 1.01719268],
        [1.85123493, 1.96436198, 1.93607634, 1.03812484, 1.47757054,
         1.29827526, 1.32091826, 1.37575186],
        [1.87548365, 1.63503737, 1.53307387, 1.81016503, 1.98840537,
         1.14883261, 1.52461467, 1.49105937],
        [1.97442634, 1.88073578, 1.98654645, 1.40322797, 1.33704814,
         1.30948304, 1.34712334, 1.68997318],
        [1.09029466, 1.14059271, 1.60518456, 1.96145458, 1.52500321,
         1.30352744, 1.76889237, 1.05815577],
        [1.00924576, 1.2678781 , 1.18180298, 1.94628611, 1.8497092 ,
         1.19566506, 1.61042227, 1.89932404],
        [1.07049536, 1.96082715, 1.80126547, 1.89507022, 1.75890745,
         1.6351449 , 1.10307299, 1.83061231]]),
 array([[1.27572293, 1.77110139, 1.33310633, 1.33785275, 1.7533493 ,
         

In [50]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            for k in range(8):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [51]:
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

0.004090547561645508


In [52]:
from pynq import allocate

In [53]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [54]:
data_input

array([1.16763793, 1.76083613, 1.6171598 , 1.60581055, 1.97313797,
       1.49387104, 1.68515268, 1.11433389, 1.1055406 , 1.6582569 ,
       1.28133038, 1.52072141, 1.20111532, 1.53132037, 1.73651009,
       1.01719268, 1.85123493, 1.96436198, 1.93607634, 1.03812484,
       1.47757054, 1.29827526, 1.32091826, 1.37575186, 1.87548365,
       1.63503737, 1.53307387, 1.81016503, 1.98840537, 1.14883261,
       1.52461467, 1.49105937, 1.97442634, 1.88073578, 1.98654645,
       1.40322797, 1.33704814, 1.30948304, 1.34712334, 1.68997318,
       1.09029466, 1.14059271, 1.60518456, 1.96145458, 1.52500321,
       1.30352744, 1.76889237, 1.05815577, 1.00924576, 1.2678781 ,
       1.18180298, 1.94628611, 1.8497092 , 1.19566506, 1.61042227,
       1.89932404, 1.07049536, 1.96082715, 1.80126547, 1.89507022,
       1.75890745, 1.6351449 , 1.10307299, 1.83061231, 1.27572293,
       1.77110139, 1.33310633, 1.33785275, 1.7533493 , 1.98442233,
       1.19604269, 1.49489847, 1.55082835, 1.91433816, 1.47165

In [55]:
input_buffer = allocate(128, np.float32)
output_buffer = allocate((8,8), np.float32)

In [56]:
np.copyto(input_buffer, data_input)

In [62]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.0031702518463134766


In [63]:
output_buffer

PynqBuffer([[19.458935, 20.084389, 20.41356 , 20.225574, 19.91514 ,
             20.261086, 19.840324, 17.803902],
            [17.268723, 17.692392, 17.971731, 17.705404, 17.731987,
             18.009583, 17.86277 , 15.987549],
            [18.706795, 19.859066, 19.642088, 19.720467, 19.769758,
             20.510658, 19.37648 , 17.764883],
            [20.468233, 21.35748 , 20.841808, 20.758278, 20.933582,
             21.363064, 20.373337, 18.695171],
            [19.933743, 20.939894, 20.459787, 20.55033 , 20.884157,
             21.580551, 20.422173, 18.856815],
            [17.96316 , 18.403349, 18.718088, 18.375107, 18.543884,
             18.566551, 18.244656, 16.660755],
            [19.449127, 19.41953 , 19.164019, 18.836687, 19.006477,
             19.267944, 18.924318, 17.075764],
            [20.952236, 21.126568, 20.989231, 21.203249, 20.906666,
             21.491829, 21.042557, 18.924456]], dtype=float32)

In [64]:
diff = np.dot(data_A, data_B) - output_buffer

In [65]:
rms_error = np.sqrt(np.mean(diff**2))

In [66]:
diff, rms_error

(PynqBuffer([[ 1.09075884e-06,  1.88122120e-06, -1.29233435e-06,
              -1.02247490e-06,  2.55436422e-07,  4.28844725e-07,
              -3.66444251e-07,  2.76026344e-06],
             [ 1.50296229e-06, -7.04755756e-08, -4.77976492e-07,
              -5.65314302e-08,  3.77106165e-07,  9.16823335e-07,
               1.44487129e-06,  5.45769012e-07],
             [-9.16361763e-07, -4.77333753e-07,  1.08711044e-06,
              -1.42183982e-06, -2.08795673e-06, -4.81946817e-07,
               3.10324200e-07,  3.58300252e-07],
             [-1.42100255e-07, -4.51815346e-07,  1.99601640e-07,
               9.83719826e-07, -2.22124086e-07, -2.30377793e-07,
              -1.05856456e-06, -4.52477114e-07],
             [-6.10885344e-07,  1.20786693e-06,  1.30311713e-06,
               1.26430223e-06, -6.46354916e-07,  1.20154311e-06,
              -4.05476168e-07,  1.28908477e-06],
             [-5.72556484e-07,  1.55162724e-07, -4.26546148e-07,
               3.30378395e-07, -4.670377

In [67]:
%%time
np.dot(data_A, data_B)

CPU times: user 98 µs, sys: 22 µs, total: 120 µs
Wall time: 467 µs


array([[19.45893587, 20.08439061, 20.41355957, 20.22557347, 19.91514041,
        20.26108594, 19.84032404, 17.80390443],
       [17.26872404, 17.69239228, 17.97173071, 17.70540423, 17.73198738,
        18.00958344, 17.86277153, 15.98754937],
       [18.70679382, 19.85906553, 19.64208902, 19.72046519, 19.76975614,
        20.51065778, 19.37648041, 17.7648834 ],
       [20.46823297, 21.35747864, 20.84180852, 20.75827888, 20.93358208,
        21.36306358, 20.37333573, 18.6951709 ],
       [19.93374191, 20.93989493, 20.45978867, 20.55033047, 20.88415653,
        21.58055235, 20.42217214, 18.85681663],
       [17.96315899, 18.40334908, 18.71808772, 18.37510714, 18.54388381,
        18.56655308, 18.24465431, 16.66075402],
       [19.44912756, 19.41952976, 19.16401961, 18.83668701, 19.00647664,
        19.26794605, 18.92431966, 17.07576389],
       [20.95223773, 21.1265691 , 20.98923228, 21.2032511 , 20.90666855,
        21.4918275 , 21.04255743, 18.92445658]])

In [69]:
rms_error

PynqBuffer(1.10141589e-06)